In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

## Importing Libraries

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import ndimage #이미지를 가지고 뭘더해주는?

print(tf.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf


## Enable Eager Mode

In [3]:
tf.enable_eager_execution()

## 1. Hyper Parameters

In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

tf.set_random_seed(777)

## Creating a Checkpoint Directory

In [5]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'minst_cnn_best'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

## MNIST/Fashion MNIST Data

In [6]:
## MNIST Dataset #########################################################
mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
##########################################################################

## Fashion MNIST Dataset #################################################
#mnist = keras.datasets.fashion_mnist
#class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
##########################################################################

## 2.<추가>Data Augmentation:rotate, shift

In [7]:
def data_augmentation(images, labels):
    aug_images = []
    aug_labels = []    
    
    for x, y in zip(images, labels):        
        aug_images.append(x)
        aug_labels.append(y) #오리지날 images, labels 먼저 저장
        
        bg_value = np.median(x) #중간값median을 저장(빈공간에 이걸로 채우려고)
        for _ in range(4): #4번돌아 데이터를 총5배로 불리겠다
            angle = np.random.randint(-15, 15, 1) #-15~15사이의 랜덤으로 1개 뽑아서 이 각도로
            rot_img = ndimage.rotate(x, angle, reshape=False, cval=bg_value) #이미지x를 랜덤뽑은각도angle로 돌려줌, 빈공간은 cval=bg_value이걸로채움
            
            shift = np.random.randint(-2, 2, 2) #숫자를 2개뽑아 가로방향,세로방향으로 옮김
            shift_img = ndimage.shift(rot_img, shift, cval=bg_value) #rot_img에 shift적용.           
            
            aug_images.append(shift_img) #리스트에 append
            aug_labels.append(y)
    #리스트에저장된걸 np.array로 리턴
    aug_images = np.array(aug_images)
    aug_labels = np.array(aug_labels)
    return aug_images, aug_labels

## Datasets

In [8]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images, train_labels = data_augmentation(train_images, train_labels) #요기/ data_augmentation 적용하여 데이터5배늘어남
    
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
    
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)    
    
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
                buffer_size=500000).batch(batch_size) #5배가 되었기 때문에 buffer_size=500000로 수정
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

## 3.<많이독특> Model Class

In [9]:
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME'):
        super(ConvBNRelu, self).__init__()
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, 
                                        padding=padding, kernel_initializer='glorot_normal')
        self.batchnorm = tf.keras.layers.BatchNormalization()
    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = self.batchnorm(layer)
        layer = tf.nn.relu(layer)
        return layer      # convolution에 쓸 conv-batchnorm-relu 묶인 layer 탄생

In [10]:
class DenseBNRelu(tf.keras.Model):
    def __init__(self, units):
        super(DenseBNRelu, self).__init__()
        self.dense = keras.layers.Dense(units=units, kernel_initializer='glorot_normal')
        self.batchnorm = tf.keras.layers.BatchNormalization()
    def call(self, inputs, training=False):
        layer = self.dense(inputs)
        layer = self.batchnorm(layer)
        layer = tf.nn.relu(layer)
        return layer #풀리커텍트에 쓸 dense-batchnorm-relu 묶인 layer탄생

In [11]:
#전체모델만들기
class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = ConvBNRelu(filters=32, kernel_size=[3, 3], padding='SAME')        
        self.pool1 = keras.layers.MaxPool2D(padding='SAME')
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], padding='SAME')
        self.pool2 = keras.layers.MaxPool2D(padding='SAME')
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3], padding='SAME')
        self.pool3 = keras.layers.MaxPool2D(padding='SAME')
        self.pool3_flat = keras.layers.Flatten()
        self.dense4 = DenseBNRelu(units=256)
        self.drop4 = keras.layers.Dropout(rate=0.4)
        self.dense5 = keras.layers.Dense(units=10, kernel_initializer='glorot_normal')
    def call(self, inputs, training=False):
        net = self.conv1(inputs)        
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

In [12]:
models = []
num_models = 5 #5개의 모델
for m in range(num_models):
    models.append(MNISTModel())

## 4. Loss Function

In [13]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=labels))    
    return loss   

## 5. Calculating Gradient

In [14]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

## 7. Caculating Model's Accuracy

In [15]:
def evaluate(models, images, labels):
    predictions = np.zeros_like(labels)
    for model in models:
        logits = model(images, training=False)
        predictions += logits
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

## 6. Optimizer

In [16]:
global_step = tf.train.get_or_create_global_step() #있으면가져오고 없으면 만들어라
lr_decay = tf.train.exponential_decay(learning_rate, global_step, 
                                      train_images.shape[0]/batch_size*num_models*5, 0.5, staircase=True) #5epoch가 되면 0.5반으로 learning_rate를 줄여라
optimizer = tf.train.AdamOptimizer(learning_rate=lr_decay)

## 8. Creating Checkpoints

In [17]:
checkpoints = []
for m in range(num_models):
    checkpoints.append(tf.train.Checkpoint(cnn=models[m]))

## 9. Training

In [18]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0    
    
    for images, labels in train_dataset:
        for model in models:
            grads = grad(model, images, labels)                
            optimizer.apply_gradients(zip(grads, model.variables), global_step=global_step)            
            loss = loss_fn(model, images, labels)
            avg_loss += loss / num_models
        acc = evaluate(models, images, labels)
        avg_train_acc += acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(models, images, labels)        
        avg_test_acc += acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    
    for idx, checkpoint in enumerate(checkpoints):
        checkpoint.save(file_prefix=checkpoint_prefix+'-{}'.format(idx))

print('Learning Finished!')
#앙상블+데이터추가+learning rate +BatchNormalization 했더니 정확도가 올라갔다!

Learning started. It takes sometime.
Epoch: 1 loss = 0.05254757 train accuracy =  0.9863 test accuracy =  0.9947
Epoch: 2 loss = 0.01288698 train accuracy =  0.9976 test accuracy =  0.9960
Epoch: 3 loss = 0.00692925 train accuracy =  0.9991 test accuracy =  0.9965
Epoch: 4 loss = 0.00416141 train accuracy =  0.9995 test accuracy =  0.9959
Epoch: 5 loss = 0.00290176 train accuracy =  0.9998 test accuracy =  0.9962
Epoch: 6 loss = 0.00146718 train accuracy =  0.9999 test accuracy =  0.9963
Epoch: 7 loss = 0.00065968 train accuracy =  0.9999 test accuracy =  0.9961
Epoch: 8 loss = 0.00049940 train accuracy =  1.0000 test accuracy =  0.9965
Epoch: 9 loss = 0.00040568 train accuracy =  1.0000 test accuracy =  0.9961
Epoch: 10 loss = 0.00036997 train accuracy =  1.0000 test accuracy =  0.9958
Epoch: 11 loss = 0.00018087 train accuracy =  1.0000 test accuracy =  0.9968
Epoch: 12 loss = 0.00006729 train accuracy =  1.0000 test accuracy =  0.9964
Epoch: 13 loss = 0.00005760 train accuracy =  1.

In [19]:
models[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_relu (ConvBNRelu)    multiple                  448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv_bn_relu_1 (ConvBNRelu)  multiple                  18752     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv_bn_relu_2 (ConvBNRelu)  multiple                  74368     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
__________